# Scraping Player Advanced Stats by Year

#### https://www.basketball-reference.com/leagues/NBA_{}_advanced.html

In [12]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from path import Path

In [158]:
def adv_stats (year):
    url = "https://www.basketball-reference.com/leagues/NBA_{}_advanced.html".format(year)
    html = urlopen(url)
    soup = BeautifulSoup(html)
    #use find_all from beautifulsoup to locate the header in the first table row (tag <tr>)
    tr_header = soup.find_all('tr')[0]
    #use get_text to extract column header names from table header (tag <th>) into a list
    headers = [th.get_text() for th in tr_header.find_all('th')]
    #drop first ranking column
    headers = headers[1:]
    #skip header row 
    tr_rows = soup.find_all('tr')[1:]
    #use get_text to extract player data from table data (tag <td>), skipping column header row
    player_data = [[td.get_text() for td in tr_rows[i].find_all('td')] for i in range(len(tr_rows))]
    #create dataframe of data and column header
    adv_stats_df = pd.DataFrame(player_data, columns=headers)
    
    return(adv_stats_df)
    
    


In [5]:
adv_stats(2019).head()

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP
0,Álex Abrines,SG,25,OKC,31,588,6.3,.507,.809,.083,...,,0.1,0.6,0.6,.053,,-2.4,-0.9,-3.4,-0.2
1,Quincy Acy,PF,28,PHO,10,123,2.9,.379,.833,.556,...,,-0.1,0.0,-0.1,-0.022,,-5.7,-0.3,-5.9,-0.1
2,Jaylen Adams,PG,22,ATL,34,428,7.6,.474,.673,.082,...,,-0.1,0.2,0.1,.011,,-3.1,-1.3,-4.4,-0.3
3,Steven Adams,C,25,OKC,80,2669,18.5,.591,.002,.361,...,,5.1,4.0,9.1,.163,,0.6,2.1,2.7,3.2
4,Bam Adebayo,C,21,MIA,82,1913,17.9,.623,.031,.465,...,,3.4,3.4,6.8,.171,,-0.6,3.6,3.0,2.4


In [13]:
def adv_stats_all():
    
    year = [2013, 2014, 2015, 2016, 2017, 2018, 2019]
    adv_stats_df_all = pd.DataFrame()
    
    for year in year:
        url = "https://www.basketball-reference.com/leagues/NBA_{}_advanced.html".format(year)
        html = urlopen(url)
        soup = BeautifulSoup(html)
        #use find_all from beautifulsoup to locate the header in the first table row (tag <tr>)
        tr_header = soup.find_all('tr')[0]
        #use get_text to extract column header names from table header (tag <th>) into a list
        headers = [th.get_text() for th in tr_header.find_all('th')]
        #drop first ranking column
        headers = headers[1:]
        #skip header row 
        tr_rows = soup.find_all('tr')[1:]
        #use get_text to extract player data from table data (tag <td>), skipping column header row
        player_data = [[td.get_text() for td in tr_rows[i].find_all('td')] for i in range(len(tr_rows))]
        #create dataframe of data and column header
        adv_stats_df = pd.DataFrame(player_data, columns=headers)
        #adds the season year
        adv_stats_df['Year']= year
        #drops duplicates and keeps the first entry, which is the total stats for the player across each team the player played in that year
        adv_stats_df.drop_duplicates(subset ="Player", 
                     keep = 'first', inplace = True) 
        adv_stats_df_all = adv_stats_df_all.append(adv_stats_df)
    
    return(adv_stats_df_all)

In [14]:
adv_stats_all_df = adv_stats_all()
adv_stats_all_df

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP,Year
0,Quincy Acy,PF,22,TOR,29,342,15.9,.632,.027,.507,...,0.7,0.4,1.1,.157,,-0.6,1.3,0.7,0.2,2013
1,Jeff Adrien,PF,26,CHA,52,713,13.4,.493,.012,.595,...,0.5,0.4,1.0,.064,,-3.0,-0.2,-3.3,-0.2,2013
2,Arron Afflalo,SF,27,ORL,64,2307,13.0,.527,.265,.246,...,1.5,0.5,2.0,.042,,-0.3,-2.0,-2.3,-0.2,2013
3,Josh Akognon,PG,26,DAL,3,9,15.3,.625,.500,.000,...,0.0,0.0,0.0,.196,,4.7,-4.9,-0.2,0.0,2013
4,Cole Aldrich,C,24,TOT,45,388,11.1,.563,.000,.250,...,0.1,0.4,0.6,.070,,-4.8,0.7,-4.0,-0.2,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,Trae Young,PG,20,ATL,81,2503,17.0,.539,.384,.330,...,2.5,0.7,3.3,.062,,1.8,-2.8,-1.1,0.6,2019
726,Cody Zeller,C,26,CHO,49,1243,17.2,.611,.064,.409,...,2.6,1.2,3.9,.150,,0.2,1.7,2.0,1.2,2019
727,Tyler Zeller,C,29,TOT,6,93,17.3,.607,.033,.600,...,0.2,0.1,0.3,.167,,-1.0,-1.2,-2.2,0.0,2019
730,Ante Žižić,C,22,CLE,59,1082,16.2,.590,.000,.399,...,1.7,0.3,2.0,.087,,-1.7,-1.4,-3.1,-0.3,2019


In [15]:
adv_stats_all_df.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', ' ',
       'OWS', 'DWS', 'WS', 'WS/48', ' ', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'Year'],
      dtype='object')

In [16]:
#TEST duplicates 
adv_stats_all_df[adv_stats_all_df['Player'] == 'Cole Aldrich']

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP,Year
4,Cole Aldrich,C,24,TOT,45,388,11.1,.563,.000,.250,...,0.1,0.4,0.6,.070,,-4.8,0.7,-4.0,-0.2,2013
9,Cole Aldrich,C,25,NYK,46,330,19.1,.620,.000,.492,...,0.6,0.6,1.2,.178,,-2.6,3.8,1.2,0.3,2014
9,Cole Aldrich,C,26,NYK,61,976,18.1,.513,.000,.213,...,0.8,1.4,2.2,.107,,-2.2,3.1,0.9,0.7,2015
5,Cole Aldrich,C,27,LAC,60,800,21.3,.626,.000,.373,...,1.4,2.0,3.5,.209,,-1.0,5.8,4.8,1.4,2016
7,Cole Aldrich,C,28,MIN,62,531,12.7,.549,.000,.256,...,0.6,0.7,1.3,.116,,-2.0,2.6,0.6,0.4,2017
5,Cole Aldrich,C,29,MIN,21,49,6.0,.340,.000,.400,...,-0.1,0.1,0.0,-0.013,,-7.0,0.0,-7.0,-0.1,2018


In [17]:
#deleted blank columns in the df 
adv_stats_all_df.columns[18]
adv_stats_all_df.drop(adv_stats_all_df.columns[18], axis=1, inplace=True)

adv_stats_all_df.columns[23]
adv_stats_all_df.drop(adv_stats_all_df.columns[23], axis=1, inplace=True)

In [18]:
adv_stats_all_df.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS',
       'DWS', 'WS', 'WS/48', 'OBPM', 'BPM', 'VORP', 'Year'],
      dtype='object')

In [19]:
adv_stats_all_df.isnull().sum()

Player    7
Pos       7
Age       7
Tm        7
G         7
MP        7
PER       7
TS%       7
3PAr      7
FTr       7
ORB%      7
DRB%      7
TRB%      7
AST%      7
STL%      7
BLK%      7
TOV%      7
USG%      7
OWS       7
DWS       7
WS        7
WS/48     7
OBPM      7
BPM       7
VORP      7
Year      0
dtype: int64

In [20]:
#7 blank rows deleted and cleaned
adv_stats_all_df_cleaned = adv_stats_all_df.dropna()
adv_stats_all_df_cleaned

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,BPM,VORP,Year
0,Quincy Acy,PF,22,TOR,29,342,15.9,.632,.027,.507,...,15.6,14.7,0.7,0.4,1.1,.157,-0.6,0.7,0.2,2013
1,Jeff Adrien,PF,26,CHA,52,713,13.4,.493,.012,.595,...,13.1,15.6,0.5,0.4,1.0,.064,-3.0,-3.3,-0.2,2013
2,Arron Afflalo,SF,27,ORL,64,2307,13.0,.527,.265,.246,...,12.1,22.5,1.5,0.5,2.0,.042,-0.3,-2.3,-0.2,2013
3,Josh Akognon,PG,26,DAL,3,9,15.3,.625,.500,.000,...,0.0,20.3,0.0,0.0,0.0,.196,4.7,-0.2,0.0,2013
4,Cole Aldrich,C,24,TOT,45,388,11.1,.563,.000,.250,...,20.6,12.7,0.1,0.4,0.6,.070,-4.8,-4.0,-0.2,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,Trae Young,PG,20,ATL,81,2503,17.0,.539,.384,.330,...,17.6,28.4,2.5,0.7,3.3,.062,1.8,-1.1,0.6,2019
726,Cody Zeller,C,26,CHO,49,1243,17.2,.611,.064,.409,...,13.2,16.3,2.6,1.2,3.9,.150,0.2,2.0,1.2,2019
727,Tyler Zeller,C,29,TOT,6,93,17.3,.607,.033,.600,...,9.5,20.1,0.2,0.1,0.3,.167,-1.0,-2.2,0.0,2019
730,Ante Žižić,C,22,CLE,59,1082,16.2,.590,.000,.399,...,13.6,18.2,1.7,0.3,2.0,.087,-1.7,-3.1,-0.3,2019


In [21]:
adv_stats_sorted = adv_stats_all_df_cleaned.sort_values(by=['Player', 'Year'])

In [183]:
adv_stats_all_df_cleaned.to_csv('../Resources/adv_stats_all.csv')

In [189]:
adv_stats_sorted.to_csv('../Resources/adv_stats_sorted.csv')

### Add the slug name to the final adv stats df 


In [22]:
#bringing in slug name from the 2013-2020_advance_data file
file_path = Path('../Resources/2013-2020_advance_data.csv')
slug_file = pd.read_csv(file_path)
slug_file.head()

,Unnamed: 0,slug,name,positions,age,team,games_played,minutes_played,player_efficiency_rating,true_shooting_percentage,...,usage_percentage,offensive_win_shares,defensive_win_shares,win_shares,win_shares_per_48_minutes,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,year
0,0,acyqu01,Quincy Acy,[<Position.POWER_FORWARD: 'POWER FORWARD'>],22,Team.TORONTO_RAPTORS,29,342,15.9,0.632,...,14.7,0.7,0.4,1.1,0.157,-0.6,1.3,0.7,0.2,2013
1,1,adrieje01,Jeff Adrien,[<Position.POWER_FORWARD: 'POWER FORWARD'>],26,Team.CHARLOTTE_BOBCATS,52,713,13.4,0.493,...,15.6,0.5,0.4,1.0,0.064,-3.0,-0.2,-3.3,-0.2,2013
2,2,afflaar01,Arron Afflalo,[<Position.SMALL_FORWARD: 'SMALL FORWARD'>],27,Team.ORLANDO_MAGIC,64,2307,13.0,0.527,...,22.5,1.5,0.5,2.0,0.042,-0.3,-2.0,-2.3,-0.2,2013
3,3,akognjo01,Josh Akognon,[<Position.POINT_GUARD: 'POINT GUARD'>],26,Team.DALLAS_MAVERICKS,3,9,15.3,0.625,...,20.3,0.0,0.0,0.0,0.196,4.7,-4.9,-0.2,0.0,2013
4,4,aldrico01,Cole Aldrich,[<Position.CENTER: 'CENTER'>],24,Team.HOUSTON_ROCKETS,30,213,7.3,0.532,...,12.5,-0.1,0.3,0.2,0.038,-5.7,1.0,-4.8,-0.1,2013


In [23]:
slug_df = slug_file[['slug','name']].copy()
slug_df = slug_df.drop_duplicates()

In [24]:
slug_df.rename(columns={'name':'Player'}, inplace = True)

In [25]:
slug_df

,slug,Player
0,acyqu01,Quincy Acy
1,adrieje01,Jeff Adrien
2,afflaar01,Arron Afflalo
3,akognjo01,Josh Akognon
4,aldrico01,Cole Aldrich
...,...,...
4433,watsopa01,Paul Watson
4436,whiteco01,Coby White
4440,willigr01,Grant Williams
4446,willini01,Nigel Williams-Goss


In [26]:
add_slug_df = adv_stats_all_df_cleaned.copy()
add_slug_df

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,BPM,VORP,Year
0,Quincy Acy,PF,22,TOR,29,342,15.9,.632,.027,.507,...,15.6,14.7,0.7,0.4,1.1,.157,-0.6,0.7,0.2,2013
1,Jeff Adrien,PF,26,CHA,52,713,13.4,.493,.012,.595,...,13.1,15.6,0.5,0.4,1.0,.064,-3.0,-3.3,-0.2,2013
2,Arron Afflalo,SF,27,ORL,64,2307,13.0,.527,.265,.246,...,12.1,22.5,1.5,0.5,2.0,.042,-0.3,-2.3,-0.2,2013
3,Josh Akognon,PG,26,DAL,3,9,15.3,.625,.500,.000,...,0.0,20.3,0.0,0.0,0.0,.196,4.7,-0.2,0.0,2013
4,Cole Aldrich,C,24,TOT,45,388,11.1,.563,.000,.250,...,20.6,12.7,0.1,0.4,0.6,.070,-4.8,-4.0,-0.2,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,Trae Young,PG,20,ATL,81,2503,17.0,.539,.384,.330,...,17.6,28.4,2.5,0.7,3.3,.062,1.8,-1.1,0.6,2019
726,Cody Zeller,C,26,CHO,49,1243,17.2,.611,.064,.409,...,13.2,16.3,2.6,1.2,3.9,.150,0.2,2.0,1.2,2019
727,Tyler Zeller,C,29,TOT,6,93,17.3,.607,.033,.600,...,9.5,20.1,0.2,0.1,0.3,.167,-1.0,-2.2,0.0,2019
730,Ante Žižić,C,22,CLE,59,1082,16.2,.590,.000,.399,...,13.6,18.2,1.7,0.3,2.0,.087,-1.7,-3.1,-0.3,2019


In [62]:
df_left = pd.merge(add_slug_df, slug_df, on='Player', how= 'left')

df_left

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,USG%,OWS,DWS,WS,WS/48,OBPM,BPM,VORP,Year,slug
0,Quincy Acy,PF,22,TOR,29,342,15.9,.632,.027,.507,...,14.7,0.7,0.4,1.1,.157,-0.6,0.7,0.2,2013,acyqu01
1,Jeff Adrien,PF,26,CHA,52,713,13.4,.493,.012,.595,...,15.6,0.5,0.4,1.0,.064,-3.0,-3.3,-0.2,2013,adrieje01
2,Arron Afflalo,SF,27,ORL,64,2307,13.0,.527,.265,.246,...,22.5,1.5,0.5,2.0,.042,-0.3,-2.3,-0.2,2013,afflaar01
3,Josh Akognon,PG,26,DAL,3,9,15.3,.625,.500,.000,...,20.3,0.0,0.0,0.0,.196,4.7,-0.2,0.0,2013,akognjo01
4,Cole Aldrich,C,24,TOT,45,388,11.1,.563,.000,.250,...,12.7,0.1,0.4,0.6,.070,-4.8,-4.0,-0.2,2013,aldrico01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3478,Trae Young,PG,20,ATL,81,2503,17.0,.539,.384,.330,...,28.4,2.5,0.7,3.3,.062,1.8,-1.1,0.6,2019,youngtr01
3479,Cody Zeller,C,26,CHO,49,1243,17.2,.611,.064,.409,...,16.3,2.6,1.2,3.9,.150,0.2,2.0,1.2,2019,zelleco01
3480,Tyler Zeller,C,29,TOT,6,93,17.3,.607,.033,.600,...,20.1,0.2,0.1,0.3,.167,-1.0,-2.2,0.0,2019,zellety01
3481,Ante Žižić,C,22,CLE,59,1082,16.2,.590,.000,.399,...,18.2,1.7,0.3,2.0,.087,-1.7,-3.1,-0.3,2019,zizican01


In [66]:
df_left.to_csv('../Resources/adv_stats_all_slug.csv')

In [68]:
df_left[df_left['Player']=='Matt Carroll']

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,USG%,OWS,DWS,WS,WS/48,OBPM,BPM,VORP,Year,slug
72,Matt Carroll,SG,32,CHA,1,6,2.9,,,,...,0.0,0.0,0.0,0.0,.029,-7.6,-15.4,0.0,2013,carroma01


In [34]:
df_left_round.isnull().sum()

Player    0
Pos       0
Age       0
Tm        0
G         0
MP        0
PER       0
TS%       0
3PAr      0
FTr       0
ORB%      0
DRB%      0
TRB%      0
AST%      0
STL%      0
BLK%      0
TOV%      0
USG%      0
OWS       0
DWS       0
WS        0
WS/48     0
OBPM      0
BPM       0
VORP      0
Year      0
slug      0
dtype: int64

In [70]:
#brough file back in to replace empty fields with zero
file_path_df_left = Path('../Resources/adv_stats_all_slug.csv')
slug_file_df_left = pd.read_csv(file_path_df_left)
slug_file_df_left.head()

,Unnamed: 0,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,USG%,OWS,DWS,WS,WS/48,OBPM,BPM,VORP,Year,slug
0,0,Quincy Acy,PF,22,TOR,29,342,15.9,0.632,0.027,...,14.7,0.7,0.4,1.1,0.157,-0.6,0.7,0.2,2013,acyqu01
1,1,Jeff Adrien,PF,26,CHA,52,713,13.4,0.493,0.012,...,15.6,0.5,0.4,1.0,0.064,-3.0,-3.3,-0.2,2013,adrieje01
2,2,Arron Afflalo,SF,27,ORL,64,2307,13.0,0.527,0.265,...,22.5,1.5,0.5,2.0,0.042,-0.3,-2.3,-0.2,2013,afflaar01
3,3,Josh Akognon,PG,26,DAL,3,9,15.3,0.625,0.500,...,20.3,0.0,0.0,0.0,0.196,4.7,-0.2,0.0,2013,akognjo01
4,4,Cole Aldrich,C,24,TOT,45,388,11.1,0.563,0.000,...,12.7,0.1,0.4,0.6,0.070,-4.8,-4.0,-0.2,2013,aldrico01


In [81]:
slug_file_df_left.drop(slug_file_df_left.columns[0], axis=1, inplace=True)

In [85]:
slug_file_df_left = slug_file_df_left.fillna(0)

In [88]:
#rounded all the data to three decimal places
slug_file_df_left = slug_file_df_left.round(3)

In [91]:
#check the empty fields were filled with zero 
slug_file_df_left[slug_file_df_left['Player']=='Matt Carroll']

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,USG%,OWS,DWS,WS,WS/48,OBPM,BPM,VORP,Year,slug
72,Matt Carroll,SG,32,CHA,1,6,2.9,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.029,-7.6,-15.4,0.0,2013,carroma01


In [90]:
slug_file_df_left.to_csv('../Resources/adv_stats_all_slug.csv')